In [2]:
import os
import cv2
import numpy as np
import pandas as pd 
from urllib.request import urlopen
import matplotlib.pyplot as plt

In [3]:
#load image to local file system from search
def save_image_from_url(name: str, url: str) -> None:
    arr = np.asarray(bytearray(urlopen(url).read()), dtype=np.uint8)
    img = cv2.imdecode(arr, -1)
    cv2.imwrite(f'{name}.png', img)

In [4]:
 def make_logo_roi(logo_path: str, background_path: str, xy_offset: tuple[int]) -> tuple:
    logo = cv2.imread(logo_path, 1)
    background = cv2.imread(background_path)
    
    x_offset, y_offset = xy_offset
    x_end, y_end = (x_offset + logo.shape[1], y_offset + logo.shape[0])
    
    rows,columns,channels = logo.shape
    roi = background[y_offset:y_offset+logo.shape[0], x_offset:x_offset+logo.shape[1]]
    
    logo_gray = cv2.cvtColor(logo, cv2.COLOR_RGB2GRAY)
    ret, mask = cv2.threshold(logo_gray, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    bg = cv2.bitwise_or(roi,roi,mask = mask)
    mask_inv = cv2.bitwise_not(logo_gray)
    fg = cv2.bitwise_and(logo,logo, mask=mask_inv)
    
    final_roi = cv2.add(bg,fg)

    return (y_offset, y_end, x_offset, x_end, final_roi)

In [15]:
def make_sample(background_path: str, roi: tuple, save_path: str) -> None:
    background = cv2.imread(background_path)
    background[roi[0] : roi[1], roi[2] : roi[3]]= roi[4]
    
    label = open(f"{save_path}.txt", "w+")
    x_center = ((roi[2] + roi[3]) / 2) / background.shape[1]
    y_center = ((roi[0] + roi[1]) / 2) / background.shape[0]
    norm_width = (roi[3] - roi[2]) / background.shape[1]
    norm_height = (roi[1] - roi[0]) / background.shape[0]
    label.write(f"0 {x_center} {y_center} {norm_width} {norm_height}\n")
    label.close()
    
    cv2.imwrite(f"{save_path}.png", background)

In [5]:
save_image_from_url('sber', 'https://sberdevices.ru/file/5c38bf50-e3f9-4c4e-8649-b773c09d6660/sber-logo.png')
save_image_from_url('background', 'https://w.forfun.com/fetch/70/700b81460b8ff6b65af31fd80bda2f7b.jpeg')

In [6]:
logo = cv2.imread('/kaggle/working/sber.png', 1)
dim = (int(logo.shape[1]/8),int(logo.shape[0]/8))
logo = cv2.resize(logo, dim, interpolation = cv2.INTER_AREA)
cv2.imwrite('resized_logo.png', logo)

True

In [7]:
background = cv2.imread('/kaggle/working/background.png')

In [16]:
roi = make_logo_roi('/kaggle/working/resized_logo.png', '/kaggle/working/background.png', (666, 777))
make_sample('/kaggle/working/background.png', roi, 'sample_1')